# Librerias necesarias: 
para relaizar el web scraping; al parecer la pagina del gobierno es una pagina estatica entonces usaremos request y la librerias beatifoulsoup para hacer el arbol html de la pagina(parsear el documento)

In [1]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
from io import BytesIO
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import openpyxl
import re
import unicodedata


# extraccion datos
con el link de la pagina accedemos a esta y extraemos su htmlatravez de un request

In [2]:
url="https://www.supersolidaria.gov.co/es/entidad/cooperativas-de-ahorro-y-credito"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# organizacion de los links 
Ahora lo que se procede a hacer es que organizamos los links de los archivos a traves para posteriormente descargarlos

In [3]:
etiqueta="a"
documentos = soup.find_all(etiqueta, href=lambda href: href and (href.endswith(".xlsx") or href.endswith(".xls") ) )
links_descarga=[documentos[i]["href"] for i in range(len(documentos))]

In [4]:
def nombre(ruta):
    nombre_archivo = ruta.rsplit("/", 1)[-1] if "/" in ruta else "archivo_descargado.xlsx"
    return nombre_archivo.strip("\"'")


def Filtrar_datos(list_links):
    Links_descarga_modif = list(map(lambda ruta: nombre(ruta).split("_"), list_links))
    indices_a_conservar = [i for i in range(len(Links_descarga_modif)) 
                        if ("estados" in Links_descarga_modif[i] and "financieros" in Links_descarga_modif[i]) or 
                        ("financiera" in Links_descarga_modif[i] and "puc" in Links_descarga_modif[i])]
    links_descarga_filtrados = [list_links[i] for i in indices_a_conservar]
    return links_descarga_filtrados

def descargar_archivo(url_base, link, carpeta_destino="Data_historico"):
    url_excel_remote = url_base + link
    try:
        respuesta = requests.get(url_excel_remote, timeout=30)
        respuesta.raise_for_status()
        nombre_archivo = nombre(url_excel_remote)
        ruta_completa = os.path.join(carpeta_destino, nombre_archivo)

        with open(ruta_completa, "wb") as f:
            f.write(respuesta.content)
        return #f"{nombre_archivo} descargado correctamente en {carpeta_destino}"
    except Exception as e:
        return f"Error descargando {url_excel_remote}: {e}"

def descargar_Archivos(lista_links, carpeta_destino="Data_historico"):
    url_base = "https://www.supersolidaria.gov.co"
    resultados = []

    # Crea la carpeta si no existe
    os.makedirs(carpeta_destino, exist_ok=True)

    with ThreadPoolExecutor(max_workers=10) as executor:
        tareas = [
            executor.submit(descargar_archivo, url_base, link, carpeta_destino)
            for link in lista_links
        ]
        for future in as_completed(tareas):
            resultados.append(future.result())

    for r in resultados:
        print(r)



In [5]:
datos_filtrados=Filtrar_datos(list_links=links_descarga)
descargar_Archivos(lista_links=datos_filtrados)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [6]:
# Codigo que resuleve para fechas 

def sin_tildes(texto):
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )

def extraer_valores_head(path):
    dataframe = pd.read_excel(path, nrows=6)
    lista_columnas_1 = dataframe['Unnamed: 0'].dropna().values.flatten().tolist()
    lista_columnas_2 = dataframe['Unnamed: 1'].dropna().values.flatten().tolist()
    lista_columnas = lista_columnas_1 + lista_columnas_2
    lista_columnas = list(map(str, lista_columnas))
    
    # Filtrar los elementos que no contengan la palabra "actualizada" (ignorando mayúsculas/minúsculas)
    lista_columnas = [col for col in lista_columnas if 'actualizada' not in sin_tildes(texto=col.lower())]
    lista_columnas = [col for col in lista_columnas if 'nota' not in sin_tildes(col.lower())]
    
    return lista_columnas


def extraer_mes(texto):
    meses_orden = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
                'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    return [palabra for palabra in texto if palabra in meses_orden]

def mes_menor(lista_meses):
    meses_orden = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio',
                'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']
    meses_unicos = list(set(lista_meses))
    meses_ordenados = sorted(meses_unicos, key=lambda mes: meses_orden.index(mes))
    return meses_ordenados[0] if meses_ordenados else None

def extraer_anos(texto):
    # Asegura que el texto sea string antes de usar regex
    if isinstance(texto, list):
        texto = " ".join(texto)
    return re.findall(r'\b\d{4}\b', texto)

def extraer_mes_ano(lista_valores):
    # Convertimos a minúsculas y separamos palabras
    texto_plano = [str(texto).lower().split() for texto in lista_valores if isinstance(texto, str)]

    # Extraer meses
    meses = [extraer_mes(palabras) for palabras in texto_plano]
    lista_meses = sum(meses, [])
    mes_primero = mes_menor(lista_meses)

    # Extraer años
    años = [extraer_anos(palabras) for palabras in texto_plano]
    lista_anos = sum(años, [])
    primer_ano = lista_anos[0] if lista_anos else None

    return mes_primero, primer_ano

def proceso_extraer_meses_años(path):
    lista_textos = extraer_valores_head(path)
    mes, año = extraer_mes_ano(lista_valores=lista_textos)
    return mes, año


In [7]:
def obtener_index_cuenta(data, valor="CUENTA"):
    """Busca la última fila donde aparece un valor en el DataFrame."""
    coincidencias = np.where(data == valor)[0]
    return coincidencias[-1]+1 if len(coincidencias) > 0 else 0

def leer_archivo_con_skip(path, valor="CUENTA"):
    preview = pd.read_excel(path, nrows=10)
    skip_index = obtener_index_cuenta(preview, valor)

    # Cargamos un par de filas para inspeccionar encabezados posibles
    data_pre = pd.read_excel(path, skiprows=skip_index, nrows=2)

    # Asegurar que todos los nombres de columna sean strings
    data_pre.columns = data_pre.columns.map(str)

    # Filtrar columnas que no empiezan por "Unnamed:"
    data_pre = data_pre.loc[:, ~data_pre.columns.str.startswith('Unnamed:')]

    # Verificamos si alguna columna parece ser numérica o contiene dígitos
    if any(col.strip().isdigit() or any(char.isdigit() for char in col) for col in data_pre.columns):
        row = preview.iloc[skip_index - 2].tolist()
        row = row[:2] + [x for x in row[2:] if not pd.isna(x)]
        data_lista = pd.read_excel(path, skiprows=skip_index)
        
        data_lista.columns = data_lista.columns.map(str)  # Asegura que los nombres sean strings
        data_lista = data_lista.drop(
            columns=[col for col in data_lista.columns if col.startswith("Unnamed:") and data_lista[col].isna().all()]
        )

        # Tomamos los nombres de columna reales que están al inicio
        columnas = list(data_lista.columns)[:2]
        row[:len(columnas)] = columnas
        data_lista.columns = row
        return data_lista

    return pd.read_excel(path, skiprows=skip_index)


def renombrar_columnas_duplicadas(df):
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        dups = cols[cols == dup].index.tolist()
        for i, idx in enumerate(dups[1:], start=1):
            cols[idx] = f"{dup}.repetida{i}"
    df.columns = cols
    return df


def clean_rows(data):
    data = data.copy()
    data.fillna(0, inplace=True)
    if 0 in data.index:
        data.drop(index=0, inplace=True)
    data = data[pd.to_numeric(data["CUENTA"], errors="coerce").notna()]
    data=renombrar_columnas_duplicadas(data)
    return data

def read_excels(list_name_excel,carpeta):
    lista_archivos = []
    for nombre in list_name_excel:
        path = os.path.join(carpeta, nombre)
        df = leer_archivo_con_skip(path)
        mes,ano=proceso_extraer_meses_años(path)
        df["MES"]=mes
        df["AÑO"]=ano
        lista_archivos.append(df)
    return lista_archivos

In [8]:
carpeta = 'Data_historico'
nombres_archivos = [f for f in os.listdir(carpeta) if os.path.isfile(os.path.join(carpeta, f))]

In [9]:
list_excel=read_excels(list_name_excel=nombres_archivos,
                        carpeta=carpeta)

c:\Users\David\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [10]:
list_excel_limpio=[]
for df in list_excel:
    if "CUENTA" in df.columns:
        list_excel_limpio.append(clean_rows(data=df))

In [11]:
Datos_2013_2025_cooperativas = pd.concat(list_excel_limpio, ignore_index=True)

# Sacar las columnas
col_mes = Datos_2013_2025_cooperativas.pop('MES')
col_ano = Datos_2013_2025_cooperativas.pop('AÑO')

# Concatenar todo junto, agregando MES y AÑO al final
Datos_2013_2025_cooperativas = pd.concat([Datos_2013_2025_cooperativas, col_mes, col_ano],axis=1)

# Renombrar columnas si es necesario
Datos_2013_2025_cooperativas.columns = list(Datos_2013_2025_cooperativas.columns[:-2]) + ['MES', 'ANO']
Datos_2013_2025_cooperativas.fillna(0, inplace=True)

C:\Users\David\AppData\Local\Temp\ipykernel_21952\4004796937.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Datos_2013_2025_cooperativas.fillna(0, inplace=True)


In [12]:
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE

# Solo columnas de texto
cols_str = Datos_2013_2025_cooperativas.select_dtypes(include=['object', 'string']).columns

# Aplica reemplazo vectorizado
Datos_2013_2025_cooperativas[cols_str] = Datos_2013_2025_cooperativas[cols_str].replace(ILLEGAL_CHARACTERS_RE, '', regex=True)


In [13]:
Datos_2013_2025_cooperativas.head()

,CUENTA,NOMBRE CUENTA,NOMBRE ENTIDAD,COOPERATIVA DE EMPLEADOS DE CAFAM,COOPERATIVA DE TRABAJADORES DE LA INDUSTRIA MILITAR,COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS,COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL,COOPERATIVA PARA EL BIENESTAR SOCIAL,COOPERATIVA FINANCIERA SAN FRANCISCO,COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA,...,MICROEMPRESAS DE ANTIOQUIA COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO CANAPRO,COOPERATIVA MULTIACTIVA UNION DE COMERCIANTES PLAZA TRINIDAD GALAN,FONDO DE EMPLEADOS DEL MINISTERIO DEL INTERIOR,COOPERATIVA AVANZA,COOPERATIVA MULTIACTIVA UNION COLOMBIANA,COOPERATIVA DE TRABAJADORES DE LA CHEC LTDA..repetida1,PRECOOPERATIVA PARA EL FOMENTO Y DESARROLLO DE LAS ACTIVIDADES MINERAS,MES,ANO
0,100000.0,ACTIVO,0,145360408682.619995,12044606237.9,338715032035.390015,156067567703.200012,98794478417.779999,10646742941.440001,70217193316.050003,...,0,0,0.0,0,0,0,0,0,diciembre,2022
1,110000.0,EFECTIVO Y EQUIVALENTE AL EFECTIVO,0,10945583122.219999,1108710206.46,15669431993.809999,26694947943.43,1918906761.24,1552585275.55,4685907214.5,...,0,0,0.0,0,0,0,0,0,diciembre,2022
2,110500.0,CAJA,0,286804632.34,10612630,822935174.32,1100000,358994133,171863116.93,124154900,...,0,0,0.0,0,0,0,0,0,diciembre,2022
3,110505.0,CAJA GENERAL,0,279124632.34,10612630,818085174.32,0,358994133,171863116.93,114654900,...,0,0,0.0,0,0,0,0,0,diciembre,2022
4,110510.0,CAJA MENOR,0,7680000,0,4850000,1100000,0,0,9500000,...,0,0,0.0,0,0,0,0,0,diciembre,2022


In [14]:
Datos_2013_2025_cooperativas_copia=Datos_2013_2025_cooperativas.copy()
len(Datos_2013_2025_cooperativas_copia)

277772

In [15]:
# Diccionario para convertir nombres de mes a número
meses_es = {
    'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
    'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
    'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
}

# Convertir nombre del mes a número
Datos_2013_2025_cooperativas_copia['mes_num'] = Datos_2013_2025_cooperativas_copia['MES'].str.lower().map(meses_es)


Datos_2013_2025_cooperativas_copia['fecha'] = pd.to_datetime(dict(year=Datos_2013_2025_cooperativas_copia['ANO'],
                                                                month=Datos_2013_2025_cooperativas_copia['mes_num'], day=1))
Datos_2013_2025_cooperativas_copia.drop(["MES", "ANO", "mes_num"], inplace=True,axis=1)
Datos_2013_2025_cooperativas_copia.head()

,CUENTA,NOMBRE CUENTA,NOMBRE ENTIDAD,COOPERATIVA DE EMPLEADOS DE CAFAM,COOPERATIVA DE TRABAJADORES DE LA INDUSTRIA MILITAR,COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS,COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL,COOPERATIVA PARA EL BIENESTAR SOCIAL,COOPERATIVA FINANCIERA SAN FRANCISCO,COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA,...,COOPERATIVA MULTIACTIVA DE APORTE Y CREDITO,MICROEMPRESAS DE ANTIOQUIA COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO CANAPRO,COOPERATIVA MULTIACTIVA UNION DE COMERCIANTES PLAZA TRINIDAD GALAN,FONDO DE EMPLEADOS DEL MINISTERIO DEL INTERIOR,COOPERATIVA AVANZA,COOPERATIVA MULTIACTIVA UNION COLOMBIANA,COOPERATIVA DE TRABAJADORES DE LA CHEC LTDA..repetida1,PRECOOPERATIVA PARA EL FOMENTO Y DESARROLLO DE LAS ACTIVIDADES MINERAS,fecha
0,100000.0,ACTIVO,0,145360408682.619995,12044606237.9,338715032035.390015,156067567703.200012,98794478417.779999,10646742941.440001,70217193316.050003,...,0,0,0,0.0,0,0,0,0,0,2022-12-01
1,110000.0,EFECTIVO Y EQUIVALENTE AL EFECTIVO,0,10945583122.219999,1108710206.46,15669431993.809999,26694947943.43,1918906761.24,1552585275.55,4685907214.5,...,0,0,0,0.0,0,0,0,0,0,2022-12-01
2,110500.0,CAJA,0,286804632.34,10612630,822935174.32,1100000,358994133,171863116.93,124154900,...,0,0,0,0.0,0,0,0,0,0,2022-12-01
3,110505.0,CAJA GENERAL,0,279124632.34,10612630,818085174.32,0,358994133,171863116.93,114654900,...,0,0,0,0.0,0,0,0,0,0,2022-12-01
4,110510.0,CAJA MENOR,0,7680000,0,4850000,1100000,0,0,9500000,...,0,0,0,0.0,0,0,0,0,0,2022-12-01


In [16]:
#eliminar datos que no parezcan en todos los años
anos_unicos = Datos_2013_2025_cooperativas_copia['fecha'].dt.year.unique() 
for ano in anos_unicos:
    data_ano = Datos_2013_2025_cooperativas_copia[Datos_2013_2025_cooperativas_copia['fecha'].dt.year == ano]
    columnas_datos = data_ano.drop(columns='fecha')
    columnas_a_eliminar = columnas_datos.columns[(columnas_datos == 0).all()]
    Datos_2013_2025_cooperativas_copia.drop(columns=columnas_a_eliminar, inplace=True)

In [17]:
Datos_2013_2025_cooperativas_copia

,CUENTA,NOMBRE CUENTA,COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS,COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL,COOPERATIVA PARA EL BIENESTAR SOCIAL,COOPERATIVA FINANCIERA SAN FRANCISCO,COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA,COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA,CAJA COOPERATIVA CREDICOOP,COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA,...,COOPERATIVA DE AHORRO Y CREDITO COOMPARTIR,COOPERATIVA DE AHORRO Y CREDITO BERLIN,COOPERATIVA DE AHORRO Y CREDITO DE AIPE,COOPERATIVA DE AHORRO Y CREDITO DE SANTANDER LIMITADA,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA DE AHORRO Y CREDITO FINANCIAFONDOS,COOPERATIVA DE AHORRO Y CREDITO DE DROGUISTAS DETALLISTAS,COOPERATIVA DE AHORRO Y CREDITO COLANTA,COOPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO AFROAMERICANA,fecha
0,100000.0,ACTIVO,338715032035.390015,156067567703.200012,98794478417.779999,10646742941.440001,70217193316.050003,141713346781.320007,84913662211.949997,21980538658.52,...,4312264780.2,44433679455.980003,28504879754.66,1641307696417.300049,383206015017,15593088956.610001,259764733974,317470642750,7498950265.05,2022-12-01
1,110000.0,EFECTIVO Y EQUIVALENTE AL EFECTIVO,15669431993.809999,26694947943.43,1918906761.24,1552585275.55,4685907214.5,14340153466.309999,6147909114.24,2294268153.21,...,119206240.86,2639964717.98,2223879542.88,244369148087.470001,7403155933,281734784.68,35860476413,27649449122,550700554.97,2022-12-01
2,110500.0,CAJA,822935174.32,1100000,358994133,171863116.93,124154900,699509200,95222896.68,11188335,...,27652700,240133447.59,247320976,12369556686.440001,2169476254,0,2000000,5735744815,474446733.3,2022-12-01
3,110505.0,CAJA GENERAL,818085174.32,0,358994133,171863116.93,114654900,699509200,95222896.68,10488335,...,27322700,239433447.59,247320976,12357373686.440001,2168001254,0,0,5732194815,473846733.3,2022-12-01
4,110510.0,CAJA MENOR,4850000,1100000,0,0,9500000,0,0,700000,...,330000,700000,0,12183000,1475000,0,2000000,3550000,600000,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277767,935000.0,OTRAS ACREEDORAS DE CONTROL,-,-,-,-,99,-,-,-,...,-,-,-,-,193496220,-,-,-,-,2014-12-01
277768,960000.0,ACREEDORAS CONTINGENTES POR CONTRA,264529148061.970001,149324333.53,54380115646.139999,2747754170,40587200705.209999,236342573973,99350653536.970001,18080159788.029999,...,8768709845.700001,40806656355,9539936205,1535086263353.699951,187107532712,9630937968,293056697041,202384285287.700012,6737912726,2014-12-01
277769,960500.0,ACREEDORAS CONTINGENTES POR CONTRA (DB),264529148061.970001,149324333.53,54380115646.139999,2747754170,40587200705.209999,236342573973,99350653536.970001,18080159788.029999,...,8768709845.700001,40806656355,9539936205,1535086263353.699951,187107532712,9630937968,293056697041,202384285287.700012,6737912726,2014-12-01
277770,980000.0,ACREEDORAS DE CONTROL POR CONTRA,26199199735.139999,315920567393.630005,7700000000,1299760000,6160000099,-,12320000000,3080000000,...,1274479903,3654956760,1386000000,80144033736.949997,8129557433,1289659000,5000000000,21405085756,2266800000,2014-12-01


In [21]:
Datos_2013_2025_cooperativas_copia.to_excel("resultados/Datos_2013_2025_cooperativas.xlsx",index=False)